In [1]:
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
import matplotlib.pyplot as plt
import seaborn as sns
from main import read_json_obj, get_da_data
from datetime import datetime
from StationData import get_station


In [2]:
setup_dict = read_json_obj("C:/Users/nilsb/PycharmProjects/Simultanity/JSON_Config.JSON")
engine = create_engine("mysql+pymysql://{user}:{pw}@{host}/{db}"
                       .format(host=setup_dict["database"]["hostname"],db=setup_dict["database"]["dbname"],
                        user=setup_dict["database"]["uname"],
                        pw=setup_dict["database"]["pwd"]))

In [3]:
db = '{Standort}_{name}'.format(Standort=setup_dict['simulation']['station'], name="quarterly19_south")
df_sud = pd.read_sql(db, engine)
df_sud.set_index(['datetime_index'], inplace=True)

In [4]:
db = '{Standort}_{name}'.format(Standort=setup_dict['simulation']['station'], name="quarterly19_e_w")
df_ew = pd.read_sql(db, engine)
df_ew.set_index(['datetime_index'], inplace=True)

In [5]:
da_df = get_da_data(year=2019)
print(f'{da_df.index.year[1]} Mittelwert des Day Ahead Börsenpreises, DE/LU')
da_mean = da_df.mean()['DA[€/MWh]']
da_df.mean()

2019 Mittelwert des Day Ahead Börsenpreises, DE/LU


DA[€/MWh]    37.67278
dtype: float64

In [6]:
revenue_sud = pd.DataFrame()
revenue_sud['DA in €/MWh'] = da_df['DA[€/MWh]']
revenue_sud['PV_rev']=df_sud['PV_output']/1000*da_df['DA[€/MWh]']/4
revenue_sud['wind_rev']=df_sud['WEA_Power']/1000*da_df['DA[€/MWh]']/4
revenue_sud['sum_rev']=(df_sud['PV_output']+df_sud['WEA_Power'])/1000 * da_df['DA[€/MWh]']/4
revenue_sud.describe()

,DA in €/MWh,PV_rev,wind_rev,sum_rev
count,35037.000000,35037.000000,34944.000000,34944.000000
mean,37.672780,7.564393,18.611894,26.196418
std,15.504588,14.416934,21.521302,28.599722
min,-90.010000,-105.383957,-128.264250,-228.599536
25%,31.060000,0.000000,2.298183,4.201616
50%,38.060000,0.000000,11.176837,17.823914
75%,46.270000,9.265454,32.108752,44.491360
max,121.460000,80.320336,133.069904,157.514126


In [7]:
revenue_ew= pd.DataFrame()
revenue_ew['DA in €/MWh'] = da_df['DA[€/MWh]']
revenue_ew['PV_rev'] = df_ew['PV_output'] / 1000 * da_df['DA[€/MWh]'] / 4
revenue_ew['wind_rev'] = df_ew['WEA_Power'] / 1000 * da_df['DA[€/MWh]'] / 4
revenue_ew['sum_rev'] = (df_ew['PV_output'] + df_ew['WEA_Power']) / 1000 * da_df['DA[€/MWh]'] / 4
revenue_ew.describe()


,DA in €/MWh,PV_rev,wind_rev,sum_rev
count,35037.000000,35037.000000,34944.000000,34944.000000
mean,37.672780,6.800495,18.611894,25.430488
std,15.504588,12.707619,21.521302,27.446612
min,-90.010000,-97.843679,-128.264250,-221.535682
25%,31.060000,0.000000,2.298183,4.273019
50%,38.060000,0.000000,11.176837,17.618293
75%,46.270000,9.058022,32.108752,43.022894
max,121.460000,73.912951,133.069904,149.603353


In [8]:
marktwert_PV = revenue_ew['PV_rev'].sum()/df_ew['PV_output'].sum()*1000*4
print(f' Mengengewichteter Marktwert PV: {round(marktwert_PV,3)} in €/MWh, {df_ew.index.year[1]} Ost-West')
marktwert_wind = revenue_ew['wind_rev'].sum()/df_ew['WEA_Power'].sum()*1000*4
print(f' Mengengewichteter Marktwert Windkraft: {round(marktwert_wind,3)} in €/MWh, {df_ew.index.year[1]} ')
marktwert_kombi = revenue_ew['sum_rev'].sum()/df_ew['power_total'].sum()*1000*4
print(f' Mengengewichteter Marktwert in Summe: {round(marktwert_kombi,3)} in €/MWh, {df_ew.index.year[1]} ')
print(f' {df_ew.index.year[1]} mittlerer DA Strompreis {round(da_mean,3)} €/MWh')

 Mengengewichteter Marktwert PV: 36.197 in €/MWh, 2019 Ost-West
 Mengengewichteter Marktwert Windkraft: 33.808 in €/MWh, 2019 
 Mengengewichteter Marktwert in Summe: 34.417 in €/MWh, 2019 
 2019 mittlerer DA Strompreis 37.673 €/MWh


In [9]:
marktwert_PV = revenue_sud['PV_rev'].sum()/df_sud['PV_output'].sum()*1000*4
print(f' Mengengewichteter Marktwert PV: {round(marktwert_PV,3)} in €/MWh, {df_ew.index.year[1]} Süd')
marktwert_wind = revenue_sud['wind_rev'].sum()/df_sud['WEA_Power'].sum()*1000*4
print(f' Mengengewichteter Marktwert Windkraft: {round(marktwert_wind,3)} in €/MWh, {df_ew.index.year[1]} ')
marktwert_kombi = revenue_sud['sum_rev'].sum()/df_sud['power_total'].sum()*1000*4
print(f' Mengengewichteter Marktwert in Summe: {round(marktwert_kombi,3)} in €/MWh, {df_ew.index.year[1]} ')
print(f' {df_ew.index.year[1]} mittlerer DA Strompreis {round(da_mean,3)} €/MWh')

 Mengengewichteter Marktwert PV: 36.473 in €/MWh, 2019 Süd
 Mengengewichteter Marktwert Windkraft: 33.808 in €/MWh, 2019 
 Mengengewichteter Marktwert in Summe: 34.539 in €/MWh, 2019 
 2019 mittlerer DA Strompreis 37.673 €/MWh


In [10]:
over = df_sud['power_total'] > df_sud['WEA_Power'].max()
e_cap = df_sud['power_total'].copy()
e_cap[over] = df_sud['WEA_Power'].max()
revenue_cap = e_cap/1000 * da_df['DA[€/MWh]']/4
loss_sud = df_sud['revenue'][over] - revenue_cap
loss_financial_perc = loss_sud.sum()/df_sud['revenue'].sum()*100
loss_energy_perc = (df_sud['power_total'].sum() - e_cap.sum())/df_sud['power_total'].sum() *100
marktwert_loss = (df_sud['power_total'][over]*da_df['DA[€/MWh]'][over]).sum()/ df_sud['power_total'][over].sum()
over_perc = sum(over)/df_sud.shape[0] *100
print('PV Ausrichtung Süd - Ergebnisse:')
print(f' {df_ew.index.year[1]} Zeitpunkte in denen g>0.5 {round(over_perc,3)} %')
print(f' {df_ew.index.year[1]} Finanzieller Verlust durch Abregelung {round(loss_financial_perc,3)} %')
print(f' {df_ew.index.year[1]} Energetischer Verlust durch Abregelung {round(loss_energy_perc,3)} %')
print(f' {df_ew.index.year[1]} Marktwert der Abgeregelten Energie {round(marktwert_loss,3)} €/MWh')

PV Ausrichtung Süd - Ergebnisse:
 2019 Zeitpunkte in denen g>0.5 16.516 %
 2019 Finanzieller Verlust durch Abregelung 8.516 %
 2019 Energetischer Verlust durch Abregelung 9.3 %
 2019 Marktwert der Abgeregelten Energie 32.653 €/MWh


In [11]:
over = df_ew['power_total'] > df_ew['WEA_Power'].max()
e_cap = df_ew['power_total'].copy()
e_cap[over] = df_ew['WEA_Power'].max()
revenue_cap = e_cap/1000 * da_df['DA[€/MWh]']/4
loss_ew = df_ew['revenue'][over] - revenue_cap
loss_financial_perc = loss_ew.sum()/df_ew['revenue'].sum()*100
loss_energy_perc = (df_ew['power_total'].sum() - e_cap.sum())/df_ew['power_total'].sum() *100
marktwert_loss = (df_ew['power_total'][over]*da_df['DA[€/MWh]'][over]).sum()/ df_ew['power_total'][over].sum()
over_perc = sum(over)/df_ew.shape[0] *100
print('PV Ausrichtung Ost West - Ergebnisse:')
print(f' {df_ew.index.year[1]} Zeitpunkte in denen g>0.5 {round(over_perc,3)} %')
print(f' {df_ew.index.year[1]} Finanzieller Verlust durch Abregelung {round(loss_financial_perc,3)} %')
print(f' {df_ew.index.year[1]} Energetischer Verlust durch Abregelung {round(loss_energy_perc,3)} %')
print(f' {df_ew.index.year[1]} Marktwert der Abgeregelten Energie {round(marktwert_loss,3)} €/MWh')

PV Ausrichtung Ost West - Ergebnisse:
 2019 Zeitpunkte in denen g>0.5 15.357 %
 2019 Finanzieller Verlust durch Abregelung 7.176 %
 2019 Energetischer Verlust durch Abregelung 7.99 %
 2019 Marktwert der Abgeregelten Energie 32.191 €/MWh


In [25]:
db = '{Standort}_{name}'.format(Standort=setup_dict['simulation']['station'], name="20_ew")
df_2020 = pd.read_sql(db, engine)
df_2020.set_index(['datetime_index'], inplace=True)
da_df = get_da_data(2020)
revenue_2020 = pd.DataFrame()
revenue_2020['DA in €/MWh'] = da_df['DA[€/MWh]']
revenue_2020['PV_rev']=df_2020['PV_output']/1000*da_df['DA[€/MWh]']/4
revenue_2020['wind_rev']=df_2020['WEA_Power']/1000*da_df['DA[€/MWh]']/4
revenue_2020['sum_rev']=(df_2020['PV_output']+df_2020['WEA_Power'])/1000 * da_df['DA[€/MWh]']/4

marktwert_PV = revenue_2020['PV_rev'].sum()/df_2020['PV_output'].sum()*1000*4
print(f' Mengengewichteter Marktwert PV: {round(marktwert_PV,3)} in €/MWh, {df_2020.index.year[1]} Ost-West')
marktwert_wind = revenue_2020['wind_rev'].sum()/df_2020['WEA_Power'].sum()*1000*4
print(f' Mengengewichteter Marktwert Windkraft: {round(marktwert_wind,3)} in €/MWh, {df_2020.index.year[1]} ')
marktwert_kombi = revenue_2020['sum_rev'].sum()/df_2020['power_total'].sum()*1000*4
print(f' Mengengewichteter Marktwert in Summe: {round(marktwert_kombi,3)} in €/MWh, {df_2020.index.year[1]} ')
da_mean = da_df.mean()['DA[€/MWh]']
print(f' {df_2020.index.year[1]} mittlerer DA Strompreis {round(da_mean,3)} €/MWh')

over = df_2020['power_total'] > df_2020['WEA_Power'].max()
e_cap = df_2020['power_total'].copy()
e_cap[over] = df_2020['WEA_Power'].max()
revenue_cap = e_cap/1000 * da_df['DA[€/MWh]']/4
loss_sud = df_2020['revenue'][over] - revenue_cap
loss_financial_perc = loss_sud.sum()/df_2020['revenue'].sum()*100
loss_energy_perc = (df_2020['power_total'].sum() - e_cap.sum())/df_2020['power_total'].sum() *100
marktwert_loss = (df_2020['power_total'][over]*da_df['DA[€/MWh]'][over]).sum()/ df_2020['power_total'][over].sum()
over_perc = sum(over)/df_2020.shape[0] *100
print('PV Ausrichtung Süd - Ergebnisse:')
print(f' {df_2020.index.year[1]} Zeitpunkte in denen g>0.5 {round(over_perc,3)} %')
print(f' {df_2020.index.year[2]} Finanzieller Verlust durch Abregelung {round(loss_financial_perc,3)} %')
print(f' {df_2020.index.year[2]} Energetischer Verlust durch Abregelung {round(loss_energy_perc,3)} %')
print(f' {df_2020.index.year[2]} Marktwert der Abgeregelten Energie {round(marktwert_loss,3)} €/MWh')

 Mengengewichteter Marktwert PV: 25.769 in €/MWh, 2020 Ost-West
 Mengengewichteter Marktwert Windkraft: 25.216 in €/MWh, 2020 
 Mengengewichteter Marktwert in Summe: 25.364 in €/MWh, 2020 
 2020 mittlerer DA Strompreis 30.47 €/MWh
PV Ausrichtung Süd - Ergebnisse:
 2020 Zeitpunkte in denen g>0.5 15.213 %
 2020 Finanzieller Verlust durch Abregelung 5.554 %
 2020 Energetischer Verlust durch Abregelung 8.417 %
 2020 Marktwert der Abgeregelten Energie 19.582 €/MWh


In [26]:
db = '{Standort}_{name}'.format(Standort=setup_dict['simulation']['station'], name="21_ew")
df_2021 = pd.read_sql(db, engine)
df_2021.set_index(['datetime_index'], inplace=True)
da_df = get_da_data(2021)
revenue_2021 = pd.DataFrame()
revenue_2021['DA in €/MWh'] = da_df['DA[€/MWh]']
revenue_2021['PV_rev']=df_2021['PV_output']/1000*da_df['DA[€/MWh]']/4
revenue_2021['wind_rev']=df_2021['WEA_Power']/1000*da_df['DA[€/MWh]']/4
revenue_2021['sum_rev']=(df_2021['PV_output']+df_2021['WEA_Power'])/1000 * da_df['DA[€/MWh]']/4

marktwert_PV = revenue_2021['PV_rev'].sum()/df_2021['PV_output'].sum()*1000*4
print(f' Mengengewichteter Marktwert PV: {round(marktwert_PV,3)} in €/MWh, {df_2021.index.year[1]} Ost-West')
marktwert_wind = revenue_2021['wind_rev'].sum()/df_2021['WEA_Power'].sum()*1000*4
print(f' Mengengewichteter Marktwert Windkraft: {round(marktwert_wind,3)} in €/MWh, {df_2021.index.year[1]} ')
marktwert_kombi = revenue_2021['sum_rev'].sum()/df_2021['power_total'].sum()*1000*4
print(f' Mengengewichteter Marktwert in Summe: {round(marktwert_kombi,3)} in €/MWh, {df_2021.index.year[1]} ')
da_mean = da_df.mean()['DA[€/MWh]']
print(f' {df_2021.index.year[1]} mittlerer DA Strompreis {round(da_mean,3)} €/MWh')

over = df_2021['power_total'] > df_2021['WEA_Power'].max()
e_cap = df_2021['power_total'].copy()
e_cap[over] = df_2021['WEA_Power'].max()
revenue_cap = e_cap/1000 * da_df['DA[€/MWh]']/4
loss_sud = df_2021['revenue'][over] - revenue_cap
loss_financial_perc = loss_sud.sum()/df_2021['revenue'].sum()*100
loss_energy_perc = (df_2021['power_total'].sum() - e_cap.sum())/df_2021['power_total'].sum() *100
marktwert_loss = (df_2021['power_total'][over]*da_df['DA[€/MWh]'][over]).sum()/ df_2021['power_total'][over].sum()
over_perc = sum(over)/df_2021.shape[0] *100
print('PV Ausrichtung Süd - Ergebnisse:')
print(f' {df_2021.index.year[1]} Zeitpunkte in denen g>0.5 {round(over_perc,3)} %')
print(f' {df_2021.index.year[1]} Finanzieller Verlust durch Abregelung {round(loss_financial_perc,3)} %')
print(f' {df_2021.index.year[1]} Energetischer Verlust durch Abregelung {round(loss_energy_perc,3)} %')
print(f' {df_2021.index.year[1]} Marktwert der Abgeregelten Energie {round(marktwert_loss,3)} €/MWh')

 Mengengewichteter Marktwert PV: 76.931 in €/MWh, 2021 Ost-West
 Mengengewichteter Marktwert Windkraft: 83.439 in €/MWh, 2021 
 Mengengewichteter Marktwert in Summe: 81.669 in €/MWh, 2021 
 2021 mittlerer DA Strompreis 96.854 €/MWh
PV Ausrichtung Süd - Ergebnisse:
 2021 Zeitpunkte in denen g>0.5 12.383 %
 2021 Finanzieller Verlust durch Abregelung 4.64 %
 2021 Energetischer Verlust durch Abregelung 7.108 %
 2021 Marktwert der Abgeregelten Energie 60.446 €/MWh


In [17]:
df_2020.describe()

,PV_output,direct,diffuse,wind_speed,wind_speed_hubheight,WEA_Power,power_total,revenue
count,35135.000000,35135.000000,35135.000000,35135.000000,35135.000000,35135.000000,35135.000000,35133.000000
mean,860.152625,84.827890,60.742471,3.684107,7.229738,2127.535537,2987.688162,19.027638
std,1459.425760,190.901501,92.424217,1.900050,3.728682,2020.412961,2703.875537,24.100297
min,0.000000,-0.000000,0.000000,0.200000,0.392482,0.000000,0.000000,-223.156389
25%,0.000000,0.000000,0.000000,2.300000,4.513549,394.465856,596.090837,2.859877
50%,0.000000,0.000000,1.087680,3.400000,6.672202,1404.926422,2207.697322,12.429277
75%,1142.797883,22.041440,97.762468,4.750000,9.321459,3779.516714,5184.568300,31.550511
max,5700.000000,963.034873,593.952626,15.900000,31.202358,5700.000000,11400.000000,176.538689
